In [13]:
import numpy as np
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
checkpoint = "EleutherAI/gpt-neo-125M"

Some top models and their sizes.

**open-source**
- GPT-Neo 125 mil
- GPT-Neo 1.3 bil (same as GPT-3 Babbage)
- GPT-2 1.5 bil
- GPT-Neo 2.7 bil
- GPT-J 6 bil
- GPT-NeoX 20 bil
- Bloom: ranges from 350m to 176 bil

**closed-source**
- GPT-3: 175 bil at biggest

# high-level api we want
https://huggingface.co/docs/transformers/internal/generation_utils

In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
generation_output = model.generate(**inputs,
                                   return_dict_in_generate=True,
                                   output_scores=True)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/chansingh/.local/lib/python3.8/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# try to get gradients

In [14]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [15]:
raw_inputs = [
    "It was good",
]
inputs = tokenizer(raw_inputs, return_tensors="pt")
# inputs['input_ids'] = inputs['input_ids'].float()
# inputs['input_ids'].requires_grad = True
print(inputs)

{'input_ids': tensor([[1026,  373,  922]]), 'attention_mask': tensor([[1, 1, 1]])}


In [33]:
model = AutoModelForCausalLM.from_pretrained(checkpoint, output_hidden_states=True)
outputs = model(**inputs)

# loss = outputs['logits'].sum()
# outputs['logits'].retain_grad()
# loss.backward(retain_graph=True)
# outputs['logits'].grad

In [34]:
outputs['logits'].shape

torch.Size([1, 3, 50257])

In [35]:
outputs['hidden_states'][0].sum()

tensor(55.4541, grad_fn=<SumBackward0>)

In [36]:
vars(model)

{'training': False,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('transformer',
               GPTNeoModel(
                 (wte): Embedding(50257, 768)
                 (wpe): Embedding(2048, 768)
                 (drop): Dropout(p=0.0, inplace=False)
                 (h): ModuleList(
                   (0): GPTNeoBlock(
                     (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                     (attn): GPTNeoAttention(
                       (attention): GPTNeoSelfAttention(
                         (attn_dropout): Dropout(p=0.0, inplace=False)
                         (resid_dropout): Dropout(p=0.0, inplace=False)
                         (k_proj): Linear(i

In [41]:
for k in model._modules:
    print(k)

transformer
lm_head


In [44]:
model._modules['lm_head']

Linear(in_features=768, out_features=50257, bias=False)

In [41]:
trans = model._modules['transformer']
out = trans(inputs['input_ids'])
for k in out:
    print(k)
h = out['hidden_states']
h[0].sum()

last_hidden_state
past_key_values
hidden_states


tensor(55.4541, grad_fn=<SumBackward0>)